In [1]:
from dotenv import load_dotenv

In [2]:
# Load .env file
load_dotenv()

True

In [11]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain.agents import initialize_agent, tool, Tool, AgentType
from langchain_community.vectorstores.chroma import Chroma
from langchain.chains import RetrievalQAWithSourcesChain
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.llms.huggingface_endpoint import HuggingFaceEndpoint
from langchain.memory import ConversationSummaryMemory
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser

In [5]:
# Loading the document
loader = PyMuPDFLoader("Advert - CEMA Internship (Software Eng. Computer Eng. Communications - Feb 2024).pdf")
documents = loader.load()

In [6]:
# Creating chunks
text_splitter = RecursiveCharacterTextSplitter()
chunks = text_splitter.split_documents(documents)

In [9]:
# Creating the vectorstore
# Persisting the model creates a sqlite db in the folder storage
vectorstore = Chroma.from_documents(chunks, embedding=OpenAIEmbeddings(), persist_directory="./storage") 
retriever = vectorstore.as_retriever()

In [17]:
template = '''
Answer questions based only on the following context:
{context}
Question: {question}
'''

prompt = ChatPromptTemplate.from_template(template)
llm = ChatOpenAI()

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

response = chain.invoke("Summarize the document and point out important parts and write in list form.")
print(response)

Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2


- The document is an advertisement for internship positions at the Center for Epidemiological Modelling and Analysis (CEMA) at the University of Nairobi.
- The internship positions available are in three tracks: Software Engineering, Computer Engineering (Artificial Intelligence & Machine Learning), and Communications.
- Interns will work closely with the CEMA team on various projects related to health communication, software development, and data analysis.
- Responsibilities include developing content for websites, social media, newsletters, and other publications; assisting in building systems for healthcare data analysis; and working on artificial intelligence and machine learning projects.
- Requirements for each track include specific educational backgrounds and interests in health-related fields.
- The internship is for a duration of six months and offers a stipend of KES 50,000 per month.
- Interested candidates need to apply online by March 4, 2024, and will be selected through